In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report
from sklearn.naive_bayes import MultinomialNB, BernoulliNB, GaussianNB
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, BaggingClassifier
from sklearn.metrics import roc_auc_score
from geopy.distance import vincenty
sns.set_style("darkgrid")

In [1]:
%%javascript
$('.nbp-app-bar').toggle()

<IPython.core.display.Javascript object>

# Read and Split the Data

In [3]:
weather=pd.read_csv('weather2.csv')
readings=pd.read_csv('readings.csv')

In [4]:
chicago=pd.merge(readings, weather, on='Date').drop(['Date', 'Month', 'Year', 'CULEX'], axis=1);
print(chicago.shape)
chicago.head()

(10506, 12)


,Virus,DayRunner,ALTER,PIPIENS,PIPIENS-RESTUANS,RESTUANS,near_hotspot,near_chompspot,avg_rain1,avg_rain2,avg_dry1,avg_dry2
0,0,149,0,0,1,0,1,4,0.714286,0.428571,0.857143,0.571429
1,0,149,0,0,0,1,1,4,0.714286,0.428571,0.857143,0.571429
2,0,149,0,0,0,1,0,3,0.714286,0.428571,0.857143,0.571429
3,0,149,0,0,1,0,2,4,0.714286,0.428571,0.857143,0.571429
4,0,149,0,0,0,1,2,4,0.714286,0.428571,0.857143,0.571429


In [5]:
# Split the data
X = chicago.drop(['Virus'], axis=1)
y = chicago['Virus']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .2, random_state=42)
X_train.shape # Notice the number of rows in the training set. This should stay constant.

(8404, 11)

In [6]:
# List the features
X.columns

Index(['DayRunner', 'ALTER', 'PIPIENS', 'PIPIENS-RESTUANS', 'RESTUANS',
       'near_hotspot', 'near_chompspot', 'avg_rain1', 'avg_rain2', 'avg_dry1',
       'avg_dry2'],
      dtype='object')

# Bring out the Neural Net!

#### Thanks to John W for this work.

In [ ]:
import tensorflow as tensor
import keras
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from keras import Sequential
from keras.layers import Convolution2D, MaxPooling2D, Dropout, Flatten, Dense
import matplotlib.pyplot as plt
%matplotlib inline

sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [ ]:
net_model = Sequential()
net_model.add(Dense(output_dim = 20,init ='uniform',activation = 'relu',input_dim=11))

net_model.add(Dense(20, activation = 'relu'))
net_model.add(Dense(20, activation = 'relu'))

net_model.add(Dense(activation="softmax", input_dim=20, units=2, kernel_initializer="uniform"))
net_model.compile(loss='categorical_crossentropy', optimizer='adam')

history = net_model.fit(X_train, to_categorical(y_train), 
            validation_data=(X_test, to_categorical(y_test)),
            epochs=6,
            batch_size=6,
            verbose = True)

In [ ]:
train_loss = history.history['loss']
test_loss = history.history['val_loss']

plt.plot(train_loss, label='Train loss')
plt.plot(test_loss, label='Test loss')
plt.legend()

In [ ]:
y_pred = net_model.predict_proba(X_test)
y_pred2 = y_pred[:,1]
print(roc_auc_score(y_test, y_pred2))
print(y_pred[:5])

In [ ]:
results=pd.DataFrame(y_pred2, columns=['column1'])
print(results.columns)
print(results.head())
results['column1'].describe()

In [ ]:
y_pred3 = y_pred[:,1].round(0)
y_pred3

In [ ]:
results=pd.DataFrame(y_pred3, columns=['column1'])
print(results.columns)
print(results.head())
results['column1'].value_counts()

## Import `test.csv` and apply the neural net